In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata

## Laden der Daten aus Datei: 

In [2]:
# open from file: 
with open("pandemie_titeldaten_oa.xml", "r", encoding="utf-8") as f2:
    result = f2.readlines()

In [ ]:
#print(result[0:500])

Parsen des XML aus Datei:

In [6]:
content = "".join(result)
#print(content[0:500])

In [20]:
xml = soup(content, "lxml")

In [24]:
#print(xml.prettify()[:500])

Heraussuchen aller Datensätze des Typs "Bibliographic" aus dem XML:

In [22]:
myquery = xml.find_all('record', {'type':'Bibliographic'})
print(len(myquery))

2895


In [25]:
#print(myquery[1].prettify())


## Suche nach benötigten Informationen in den einzelnen Datensätzen: 

In [26]:
#Funktion für Titeldaten in MARC21

def parse_record_marc(item):

    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(item)))
    
    #idn
    idn = xml.findall("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'N/A' 
        
    
    #Titel 
    title = xml.findall("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    title2 = xml.findall("marc:datafield[@tag = '245']/marc:subfield[@code = 'b']", namespaces=ns)
    
    if title and not title2:
        titletext = title[0].text
    elif title and title2:     
        titletext = title[0].text + " " + title2[0].text   #Hier wird extra nur mit einem Leerzeichen getrennt statt eines Separators
    else:
        titletext = "N/A"
    
    
    #date
    date = xml.findall("marc:datafield[@tag = '264']/marc:subfield[@code = 'c']", namespaces=ns)
    date2 = xml.findall("marc:controlfield[@tag = '008']", namespaces=ns)
    if date:
        date = date[0].text
    elif date2:
        date = date2[0].text
        date = date[7:11]
    else:    
        date = 'N/A'
          
    #lang:
    lang = xml.findall("marc:datafield[@tag = '041']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        lang = lang[0].text
    except:
        lang = 'N/A'
    
    
    #URN: 
    urn = "N/A"
    for urn in xml.findall("marc:datafield[@tag = '024']/marc:subfield[@code = 'a']", namespaces=ns):
        if urn.text.startswith("urn:nbn"):
            urn = urn.text
        else: 
            urn = "N/A"
    
    
    meta_dict = {"idn":idn, "title": titletext, "date":date, "lang":lang, "urn":urn} 
               
    return meta_dict
    

In [27]:
output = [parse_record_marc(record) for record in myquery]

In [28]:
df = pd.DataFrame(output)
df

,idn,title,date,lang,urn
0,1317002237,Ängste der Allgemeinbevölkerung in Zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970
1,131701071X,Auswirkungen der COVID-19-Pandemie auf die Pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525
2,1319384919,Auswirkungen der SARS-CoV-2-Pandemie auf die B...,2024,ger,urn:nbn:de:bvb:19-331095
3,1319553818,Beeinträchtigung des Gesichtsfeldes durch das ...,2024,ger,urn:nbn:de:101:1-2024021604063338639840
4,131589002X,Der Serotonin-Wiederaufnahmehemmer Fluoxetin...,2024,ger,urn:nbn:de:bvb:20-opus-347190
...,...,...,...,...,...
2890,1275808069,Zur Rolle von Unterhaltungsmagazinen in der Pa...,2022,ger,urn:nbn:de:101:1-2022121621092079245245
2891,1308344058,Zur seelischen Gesundheit von Studierenden – E...,2023,ger,urn:nbn:de:101:1-2023110209541661089956
2892,1276936842,"Zusammenhänge zwischen sozialer Herkunft, Unte...",2022,ger,urn:nbn:de:101:1-2022123109195820053242
2893,1280215933,Zuteilung knapper Intensivkapazitäten in der P...,2022,ger,urn:nbn:de:101:1-2023020621022586089746


In [29]:
df.to_hdf("pandemie_titeldaten_oa.h5", key="df")